## Obs Station Metadata

In [1]:
# %matplotlib nbagg

import re

from obspy import UTCDateTime, read_inventory
from obspy.clients.nrl import NRL
from obspy.io.xseed import Parser
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.fdsn import Client as FDSNClient

In [2]:
# could be replaced with a local download of NRL
nrl = NRL()

## Example to find out correct keys for given sensor in NRL

### Sensor

In [3]:
print(nrl.sensors)

manufacturer = input("\nChoose manufacturer: ");print("\n_______________________________")

Select the sensor manufacturer (42 items):
  'ASIR', 'CEA-DASE', 'CME (now R-Sensors)', 'Chaparral Physics',
  'DTCC (manuafacturers of SmartSolo)', 'EQMet', 'Eentec', 'GEObit',
  'GEOsig', 'GaiaCode', 'Gem', 'Generic', 'Geo Space/OYO', 'Geodevice'
  'Geotech', 'Guralp', 'HGS Products', 'High Tech', 'Hyperion',
  'IESE', 'Johnson Infrasound', 'Kinemetrics', 'LaHusen', 'Lennartz',
  'Lunitek', 'Magseis Fairfield', 'Metrozet', 'Nanometrics',
  'R-Sensors (previously listed as CME', 'REF TEK', 'RTClark', 'SARA'
  'Seismo Wave', 'SensorNederland', 'Sercel/Mark Products',
  'Silicon Audio', 'SolGeo', 'Sprengnether (now Eentec)',
  'Streckeisen', 'Sunfull', 'TDG', 'iTem'

Choose manufacturer: Nanometrics

_______________________________


In [4]:
print(nrl.sensors[manufacturer])

sensor = input("\nChoose sensor: ");print("\n_______________________________")

Select the Nanometrics sensor model (14 items):
  'Meridian Compact Posthole', 'Meridian Posthole',
  'Titan Accelerometer', 'Trillium 120 Horizon', 'Trillium 120P/PA',
  'Trillium 120Q/QA,PH,BH (Vault, Posthole or Borehole)',
  'Trillium 240', 'Trillium 360', 'Trillium 40',
  'Trillium All-Terrain', 'Trillium Cascadia Accelerometer',
  'Trillium Cascadia seismometer', 'Trillium Compact 120 (Vault,
  Posthole, OBS)', 'Trillium Compact 20 (Vault, Posthole, OBS)'


KeyboardInterrupt: Interrupted by user

In [ ]:
print(nrl.sensors[manufacturer][sensor])

sensitivity = input("\nChoose sensitivity: ");print("\n_______________________________")

In [ ]:
print(nrl.sensors[manufacturer][sensor][sensitivity])

generation = str(input("\nChoose generation: "));print("\n_______________________________")

In [5]:
[manufacturer,sensor,sensitivity]

NameError: name 'sensor' is not defined

In [ ]:
# nrl.sensors[manufacturer][sensor][sensitivity][generation]
nrl.sensors[manufacturer][sensor][sensitivity]

### Data Logger

In [ ]:
print(nrl.dataloggers)

datalogger = input("\nChoose datalogger: ");print("\n_______________________________")

In [ ]:
print(nrl.dataloggers[datalogger])

model = input("\nChoose datalogger model: ");print("\n_______________________________")

In [ ]:
print(nrl.dataloggers[datalogger][model])

gain = input("\nChoose datalogger gain: ");print("\n_______________________________")

In [ ]:
print(nrl.dataloggers[datalogger][model][gain])

sampling_rate = input("\nChoose datalogger sampling rate: ");print("\n_______________________________")

In [ ]:
print(nrl.dataloggers[datalogger][model][gain][sampling_rate])

In [ ]:
[datalogger, model, gain, sampling_rate]

### Response

In [ ]:
response = nrl.get_response(
    datalogger_keys=[datalogger, model, gain, sampling_rate],
    sensor_keys=[manufacturer,sensor,sensitivity]
    )

In [ ]:
response.plot(0.001);

## More Details on Station

In [ ]:
net = input("\nEnter network: ");print("\n_______________________________")

sta = input("\nEnter station: ");print("\n_______________________________")

loc = input("\nEnter location: ") or '';print("\n_______________________________")

serial_number = input("\nEnter serial number: ");print("\n_______________________________")

outpath = input("\nEnter path of output file: ");print("\n_______________________________")

location = input("\nSpecify site coordinates (y/n)? ")

if location == "y" or location == "yes":
    lat = input("Enter latitude: ")
    lon = input("Enter longitude: ")
    ele = input("Enter elevation: ")
    
else:
    lat, lon, ele = 0.0, 0.0, 0.0

outfile = f"{serial_number}_{sta}.xml"

In [ ]:
channel1 = Channel(code='HHZ', 
                   location_code=loc, 
                   latitude=lat, 
                   longitude=lon,
                   elevation=ele, 
                   depth=0,
#                    azimuth=0,
#                    dip=-90,
                   sample_rate=sampling_rate,
                   response=response,
                  )

channel2 = Channel(code='HHN', 
                   location_code=loc, 
                   latitude=lat, 
                   longitude=lon,
                   elevation=ele, 
                   depth=0,
#                    azimuth=0,
#                    dip=0,
                   sample_rate=sampling_rate,
                   response=response,
                  )

channel3 = Channel(code='HHE', 
                   location_code=loc, 
                   latitude=lat, 
                   longitude=lon,
                   elevation=ele, 
                   depth=0,
#                    azimuth=90,
#                    dip=0,
                   sample_rate=sampling_rate,
                   response=response,
                  )

In [ ]:
site = Site(name=loc)


station = Station(code=sta, 
                  latitude=lat, 
                  longitude=lon,
                  elevation=ele,
                  channels=[channel1,channel2,channel3],
                  site=site,
                 )

network = Network(code=net,
                  stations=[station],
                 )


inv = Inventory(networks=[network], 
                source='LMU',
               )


if outpath[-1] == "/":
    outpath = outpath[:-1]

### Write XML output

In [ ]:
inv.write(f"{outpath}/{outfile}", format='STATIONXML')

### Check if output is readable

In [ ]:
try:
    read_inventory(f"{outpath}/{outfile}")
    print("\n DONE")
except:
    print("\n Something went wrong! File: {outpath}/{outfile} could not be loaded!")